In [1]:
versioninfo()

using Pkg

Pkg.activate(pwd())
Pkg.instantiate()
Pkg.status()



Julia Version 1.11.7
Commit f2b3dbda30a (2025-09-08 12:10 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 24 × Intel(R) Core(TM) i9-9920X CPU @ 3.50GHz
  WORD_SIZE: 64
  LLVM: libLLVM-16.0.6 (ORCJIT, skylake-avx512)
Threads: 24 default, 0 interactive, 12 GC (on 24 virtual cores)
Environment:
  JULIA_NUM_THREADS = auto


  Activating project at `~/Documents/Codes/Julia/ICAmm`


Project ICAmm v0.1.0
Status `~/Documents/Codes/Julia/ICAmm/Project.toml`
  [13e28ba4] AppleAccelerate v0.4.1
  [6e4b80f9] BenchmarkTools v1.6.0
⌃ [052768ef] CUDA v5.8.2
⌃ [992eb4ea] CondaPkg v0.2.31
⌃ [a93c6f00] DataFrames v1.7.0
⌃ [31c24e10] Distributions v0.25.120
  [46192b85] GPUArraysCore v0.2.0
  [7a12625a] LinearMaps v3.11.4
  [bdcacae8] LoopVectorization v0.12.172
  [23992714] MAT v0.10.7
⌃ [dde4c033] Metal v1.6.2
  [6f286f6a] MultivariateStats v0.10.3
⌃ [91a5bcdd] Plots v1.40.20
  [21216c6a] Preferences v1.5.0
⌅ [08abe8d2] PrettyTables v2.4.0
  [438e738f] PyCall v1.96.4
⌃ [6099a3de] PythonCall v0.9.27
  [1fd47b50] QuadGK v2.11.2
⌃ [f2b01f46] Roots v2.2.8
  [10745b16] Statistics v1.11.1
⌃ [6aa20fa7] TensorOperations v5.2.0
  [a759f4b9] TimerOutputs v0.5.29
  [bc48ee85] Tullio v0.3.8
  [8149f6b0] WAV v1.2.0
  [37e2e46d] LinearAlgebra v1.11.0
  [9a3f8284] Random v1.11.0
Info Packages marked with ⌃ and ⌅ have new versions available. Those with ⌃ may be upgradable, but those with ⌅ 

In [2]:
using LinearAlgebra, Random, Plots, BenchmarkTools, MultivariateStats
using WAV, MAT, Statistics

# using ICAmm
using Distributions, LoopVectorization
using TimerOutputs
using Base.Threads

size :(20 64246)


# MM Algorithm for ICA

In [4]:

using LinearAlgebra, Random, Metal, CUDA, Distributions, LoopVectorization, AppleAccelerate

function create_array(data::Union{AbstractMatrix{T}, AbstractVector{T}}, use_gpu::Bool=false, use_metal::Bool=false) where T <: AbstractFloat
    if use_gpu
        @assert isdefined(Main, :CUDA) "CUDA.jl is not loaded. Please install and load CUDA.jl."
        return isa(data, CuArray) ? data : CuArray(data)
    elseif use_metal
        @assert isdefined(Main, :Metal) "Metal.jl is not loaded. Please install and load Metal.jl."
        return isa(data, MtlArray) ? data : MtlArray(data)
    else
        return isa(data, Array) ? data : Array(data)
    end
end

# Define the icastruct
mutable struct icastruct{T <: AbstractFloat, A <: AbstractArray{T}}
    X :: A
    W :: A
    Y :: A
    M_storage1 :: A
    M_storage2 :: A
    M_storage3 :: A
    M_storage4 :: A
    M_storage5 :: A
    E_storage :: AbstractVector{T}
    G :: A
    G_old :: A
    psiY :: A
    psidY :: A
    direction :: A
    I_storage :: A
    h :: A
end

function icastruct(X::AbstractMatrix{T}, use_gpu::Bool=false, use_metal::Bool=false) where T <: AbstractFloat
    # Convert input matrix to the appropriate device (CPU/GPU)
    X_device = create_array(X, use_gpu, use_metal)
    m, n = size(X_device)
    @assert n ≥ m "Expect more samples (columns) than dimensions (rows) in X"
    
    # Initialize other fields with consistent types
    W = create_array(Matrix{T}(I, m, m), use_gpu, use_metal)
    M_storage1 = create_array(zeros(T, m, m), use_gpu, use_metal)
    M_storage2 = create_array(zeros(T, m, m), use_gpu, use_metal)
    M_storage3 = create_array(zeros(T, m, m), use_gpu, use_metal)
    M_storage4 = create_array(zeros(T, m, n), use_gpu, use_metal)
    M_storage5 = create_array(zeros(T, m, n), use_gpu, use_metal)
    I_storage = create_array(Matrix{T}(I, m, m), use_gpu, use_metal)
    E_storage = create_array(zeros(T, m), use_gpu, use_metal)  # Reshape as 2D array
    
    Y = similar(X_device)
    Y .= 0

    return icastruct(X_device, 
        W, 
        Y,
        M_storage1, 
        M_storage2, 
        M_storage3, 
        M_storage4, 
        M_storage5, 
        E_storage,
        similar(W),
        similar(W),
        similar(X),
        similar(X),
        similar(W),
        I_storage,
        similar(W))
end

# eltype(::icastruct{T, A, B}) where {T, A, B} = T


icastruct

In [16]:
function threaded_tanh!(Z::AbstractMatrix, Y::AbstractMatrix)
    @threads for j in 1:size(Y, 2)
        @turbo for i in 1:size(Y, 1)
            Z[i, j] = tanh(Y[i, j]/2)
        end
    end
end

function whitening(X)
    cov_X = cov(X', corrected=false)
    eigenvalues, eigenvectors = eigen(cov_X)
    D_inv_sqrt = Diagonal(1 ./ sqrt.(eigenvalues))
    W = eigenvectors * D_inv_sqrt * eigenvectors'
    X_whitened = W * X
    return X_whitened, W
end

function score_cpu!(icas::icastruct{T, M}) where {T <: AbstractFloat, M <: AbstractMatrix{T}}
#     @. ica.M_storage3 .= ica.Y ./ T(2)
    threaded_tanh!(icas.psiY, icas.Y)
end

function score_gpu!(icas::icastruct{T, M}) where {T<:AbstractFloat, M<:AbstractMatrix{T}}
    half = eltype(icas.Y)(0.5f0)
    @. icas.psiY = tanh(half * icas.Y)
end

function _logabsdet_any(A::AbstractMatrix{T}) where {T<:AbstractFloat}
    F = lu(A)
    U = UpperTriangular(F.U)
    s = zero(T)
    @inbounds @tturbo for i in 1:size(U,1)
        s += log(abs(U[i,i]))
    end
    return s
end

function _log2cosh_sums_threaded!(buf::AbstractMatrix{T}, Y::AbstractMatrix{T}) where {T<:AbstractFloat}
    nt = Threads.nthreads()
    s_abs_t  = zeros(T, nt)
    s_tail_t = zeros(T, nt)
    @inbounds Threads.@threads for j in axes(Y,2)
        tid = Threads.threadid()
        sA = zero(T); sT = zero(T)
        @simd for i in axes(Y,1)
            a = abs(Y[i,j])
            sA += a
            t = log1p(exp(-a))  # stable
            sT += t
            buf[i,j] = t
        end
        s_abs_t[tid]  += sA
        s_tail_t[tid] += sT
    end
    return sum(s_abs_t), sum(s_tail_t)
end

function loss(icas::icastruct{T,M}, Y::M, W::M) where {T<:AbstractFloat,M<:AbstractMatrix{T}}
    n = size(Y, 2)
    log_det = log(abs(det(W)))
    s_abs, s_tail = _log2cosh_sums_threaded!(icas.M_storage4, Y)
    logcosh_sum = s_abs + 2*s_tail
    return -log_det + logcosh_sum / n
end

function gradient(icas::icastruct{T, M}) where {T<:AbstractFloat, M<:AbstractMatrix{T}}
    m, n = size(icas.Y)
    copyto!(icas.M_storage1, icas.I_storage)
    α = one(T) / T(n)
    mul!(icas.M_storage1, icas.psiY, transpose(icas.Y), -α, one(T))
    return icas.M_storage1
end

# Generate data for testing
function generate_data(m::Int, n::Int; use_gpu::Bool=false, use_metal::Bool=false)
    S1 = create_array(randn(m, n), use_gpu, use_metal)
    B = create_array(randn(m, m), use_gpu, use_metal)
    return B * S1
end

using TimerOutputs

function ica_ken(X::AbstractMatrix{T};
                 maxiter::Int = 1000, 
                 tol = 1e-6, 
                 verbose::Bool = false, 
                 W_warmStart::Union{AbstractMatrix{T}, Nothing} = nothing,
                 nesterov::Bool = true) where {T <: AbstractFloat}

    use_metal = isa(X, MtlMatrix)
    use_gpu = isa(X, CuArray)

    if use_metal || use_gpu
#         println("using gpu")
        tol = Float32(tol)
        ONE = 1f0
        TWO = 2f0
        HALF = 0.5f0
        sTWO = sqrt(2f0)
        sHALF = sqrt(1/(2f0))
    else
        tol = T(tol) 
        ONE = T(1)
        TWO = T(2)
        HALF = T(0.5)
        sTWO = sqrt(T(2))
        sHALF = sqrt(1/T(2))
#         println("using cpu")
    end

    icas = icastruct(X, use_gpu, use_metal)
    m, n = size(icas.X)
    log_liks = create_array(zeros(T, 0), use_gpu, use_metal)

    gradient_norm = ONE
    current_loss = nothing
    final_loss = nothing
    
    D = similar(icas.W)
    mul!(D, X, transpose(X), HALF/T(n), 0)
    D = sHALF.*D
    c = -sHALF/T(n)
    XT2n = c.*transpose(icas.X)   # XT2n/
        
    icas.G_old .= similar(icas.W)
    icas.G .= similar(icas.W)
    W_old = similar(icas.W)
    copyto!(W_old, icas.W)
    W_prev = similar(icas.W)
    
    t_k = T(1)
    t_new = T(1)
    if W_warmStart != nothing
        copyto!(icas.W, W_warmStart)
    end
    
    mul!(icas.Y, icas.W, icas.X)
    A_cpu = Matrix(icas.M_storage1)
    
    niters = maxiter
    for iter in 1:maxiter
        
        if use_metal || use_gpu
            score_gpu!(icas)
        else
            score_cpu!(icas)
        end
        
        if mod(iter, Int(5)) == 0
            icas.G .= gradient(icas)
            gradient_norm = norm(icas.G, Inf)
            if gradient_norm < tol
                niters = iter
                break
            end
            
            if verbose
                current_loss = loss(icas, icas.Y, icas.W)
                println("iteration ", iter, ", gradient norm: ", gradient_norm,
                    ", loglikelihood: ", current_loss)
            end
        end
        
        ## Calculating C^T = 1/(4n)*A*X^T = 0.5*XXT/(2n)-psiY*XT2n = 0.5*D-psiY*XT2n
        ## Calculating icas.M_storage2 = D^(-1/2)*C^T = sqrt(T(2)) C^T
        mul!(icas.M_storage2, icas.W, D)
        mul!(icas.M_storage2, icas.psiY, XT2n, ONE, ONE)  # C^T * D^(-1/2)
        
        icas.M_storage1 = transpose(icas.M_storage2)  ## M = D^(-1/2) C
        
        ## using SVD
        copyto!(A_cpu, icas.M_storage1)
        S = svd(A_cpu)
        copyto!(icas.M_storage1, create_array(S.U, use_gpu, use_metal))
        copyto!(icas.M_storage2, create_array(S.V, use_gpu, use_metal))
        copyto!(icas.E_storage, sqrt.(S.S .^ TWO .+ ONE) .+ S.S)
        # V[Sigma + sqrt(Sigma^2 + 1)]U^T D^{-1/2}
        Dia = Diagonal(icas.E_storage) 
        mul!(icas.W, icas.M_storage2, Dia)
        mul!(icas.M_storage3, icas.W, transpose(icas.M_storage1))
#         mul!(icas.W, icas.M_storage3, icas.D_inv_sqrt)
        icas.W .= sTWO.*icas.M_storage3

        if nesterov
            t_new = (ONE + sqrt(ONE + TWO*TWO * t_k^TWO)) / TWO
        else
            t_new = t_k
        end
        
        θ = (t_k - ONE) / t_new
        icas.M_storage1 .= icas.W - W_old             # ΔW = W - W_old
        copyto!(W_old, icas.W)                        # W_old = W
         @. icas.W .= icas.W + θ * icas.M_storage1    # fused update

        mul!(icas.Y, icas.W, icas.X)      # matrix multiply on GPU
        t_k = t_new
    end
#     final_loss = loss(icas, icas.Y, icas.W)
    return icas.W, niters, final_loss
end

ica_ken (generic function with 1 method)

In [10]:
# filename = "./faster-ica/examples/eeg.mat"
filename = "./faster-ica/examples/fmri.mat"

file = matopen(filename)
X = read(file, "X")
close(file)
X_mean = mean(X, dims=2)
X = X .- X_mean
X_whitened, _ = whitening(X)
n_features, n_samples = size(X_whitened)
println("size :","(", n_features, " ", n_samples, ")")


size :(20 64246)


In [17]:
# fmri
X_whitened_cpu = Float32.(X_whitened)
X_whitened_GPU = cu(X_whitened_cpu)
tol = 1e-4
@benchmark W_GPU, nters_GPU, loss_GPU = ica_ken($X_whitened_GPU,
    maxiter=500,
    nesterov=true,
    tol=tol,
    verbose=false)


BenchmarkTools.Trial: 119 samples with 1 evaluation per sample.
 Range (min … max):  36.682 ms … 73.955 ms  ┊ GC (min … max): 0.00% … 16.24%
 Time  (median):     38.694 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   42.090 ms ±  7.283 ms  ┊ GC (mean ± σ):  3.95% ±  6.21%

  ▄█                                                           
  ██▆▅▆▆▃▆▃▁▁▃▄▁▁▁▃▃▃▁▃▃▁▁▁▁▃▁▁▁▁▃▄▄▃▁▁▄▃▃▁▃▃▁▁▁▃▃▁▁▁▁▁▁▁▁▁▁▃ ▃
  36.7 ms         Histogram: frequency by time        63.5 ms <

 Memory estimate: 13.00 MiB, allocs estimate: 70465.

In [20]:
tol = 1e-4
@benchmark W_cpu, nters_cpu, loss_cpu = ica_ken($X_whitened_cpu,
    maxiter=500,
    nesterov=true,
    tol=tol,
    verbose=false)


BenchmarkTools.Trial: 22 samples with 1 evaluation per sample.
 Range (min … max):  217.108 ms … 274.581 ms  ┊ GC (min … max): 0.00% … 12.78%
 Time  (median):     230.920 ms               ┊ GC (median):    0.88%
 Time  (mean ± σ):   234.020 ms ±  12.989 ms  ┊ GC (mean ± σ):  1.43% ±  2.68%

  ▃        ▃   █  ▃                                              
  █▁▁▇▁▁▁▇▁█▇▇▁█▁▇█▁▇▇▁▁▁▁▁▇▇▁▁▁▁▇▁▁▇▁▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▇ ▁
  217 ms           Histogram: frequency by time          275 ms <

 Memory estimate: 32.01 MiB, allocs estimate: 12997.

In [21]:
tol = 1e-4
@benchmark W, nters, _ = ica_ken($X_whitened,
    maxiter=500,
    nesterov=true,
    tol=tol,
    verbose=false)

BenchmarkTools.Trial: 13 samples with 1 evaluation per sample.
 Range (min … max):  361.198 ms … 481.603 ms  ┊ GC (min … max): 1.00% … 19.16%
 Time  (median):     379.437 ms               ┊ GC (median):    0.62%
 Time  (mean ± σ):   385.854 ms ±  30.718 ms  ┊ GC (mean ± σ):  2.28% ±  5.19%

  ▁ ▁█  ▁▁ █▁  ▁ ▁  ▁                                         ▁  
  █▁██▁▁██▁██▁▁█▁█▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  361 ms           Histogram: frequency by time          482 ms <

 Memory estimate: 62.73 MiB, allocs estimate: 13549.

## Running in Python

In [22]:
function _log2cosh_sums_threaded!(buf::AbstractMatrix{T}, Y::AbstractMatrix{T}) where {T<:AbstractFloat}
    nt = Threads.nthreads()
    s_abs_t  = zeros(T, nt)
    s_tail_t = zeros(T, nt)
    @inbounds Threads.@threads for j in axes(Y,2)
        tid = Threads.threadid()
        sA = zero(T); sT = zero(T)
        @simd for i in axes(Y,1)
            a = abs(Y[i,j])
            sA += a
            t = log1p(exp(-a))  # stable
            sT += t
            buf[i,j] = t
        end
        s_abs_t[tid]  += sA
        s_tail_t[tid] += sT
    end
    return sum(s_abs_t), sum(s_tail_t)
end


function loss_python(Y, W)
    n = size(Y, 2)
    M_storage4 = similar(Y)
    log_det = _logabsdet_any(W)   # your existing routine
    s_abs, s_tail = _log2cosh_sums_threaded!(M_storage4, Y)
    logcosh_sum = s_abs + 2*s_tail
    return -log_det + logcosh_sum / n
end



loss_python (generic function with 1 method)

In [23]:
using PyCall

sys = pyimport("sys"); 
pushfirst!(PyVector(sys."path"),
           "/home/xunjianli/Documents/Codes/Julia/ICAmm/faster-ica")
pyimport("ml_ica")

py"""
import os
import numpy as np
from scipy.io import loadmat

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

from ml_ica.tools import whitening, callback
from ml_ica.algorithms import (
    picard, simple_quasi_newton_ica, truncated_ica, trust_region_ica
)

def _cb_get(cb, key):
    try:
        return cb[key]
    except Exception:
        return None

def run_ica_collect(mat_path, tol=1e-4, max_iter=250, double=True, out_png=None):
    X = loadmat(mat_path)['X']
    X = X - X.mean(axis=1, keepdims=True)
    X, _ = whitening(X)

    algos  = [truncated_ica, trust_region_ica, simple_quasi_newton_ica, picard]
    names  = ['Truncated Newton ICA', 'Trust region ICA', 'Simple quasi-Newton ICA', 'Picard']

    results = {}

    if out_png is not None:
        plt.figure()

    for algo, name in zip(algos, names):
        if name == 'Trust region ICA':
            continue

        cb = callback(['timing', 'gradient_norm', 'loss', 'objective'])

        if double:
            X_copy = X.copy()
        else:
            X_copy = X.astype(np.float32, copy=True)

        Y, W = algo(X_copy, verbose=1, callback=cb, tol=tol, max_iter=max_iter)

        times = _cb_get(cb, 'timing') or []
        grads = _cb_get(cb, 'gradient_norm') or []
        loss  = _cb_get(cb, 'loss')
        if loss is None:
            loss = _cb_get(cb, 'objective')

        times = np.asarray(times, dtype=float) if len(times)>0 else None
        grads = np.asarray(grads, dtype=float) if len(grads)>0 else None
        loss  = np.asarray(loss,  dtype=float) if (loss is not None and len(loss)>0) else None

        results[name] = {
            'Ys': Y,
            'Ws': W,
            'times': times,
            'grads': grads,
            'loss' : loss,
            'iters': int(len(times)) if times is not None else 0
        }

        if out_png is not None and (times is not None) and (grads is not None):
            plt.semilogy(times, grads, label=name)

    if out_png is not None:
        plt.legend(); plt.xlabel('Time (sec)'); plt.ylabel('Gradient norm')
        plt.tight_layout(); plt.savefig(out_png, dpi=150); plt.close()

    return results
"""


In [24]:

ml = pyimport("ml_ica")
root = "/home/xunjianli/Documents/Codes/Julia/ICAmm/faster-ica"
# matfile = joinpath(root, "examples", "eeg.mat")
matfile = joinpath(root, "examples", "fmri.mat")
res_py = py"run_ica_collect"(matfile, 1e-4, 250, true, "/tmp/ica_grad.png")  ## choosing double precision


Dict{Any, Any} with 3 entries:
  "Simple quasi-Newton ICA" => Dict{Any, Any}("grads"=>[0.612867, 0.945988, 1.1…
  "Picard"                  => Dict{Any, Any}("grads"=>[0.612867, 0.945988, 0.7…
  "Truncated Newton ICA"    => Dict{Any, Any}("grads"=>[0.612867, 0.459176, 0.3…

In [32]:
using DataFrames, PrettyTables, Printf

In [33]:
names  = ["Truncated Newton ICA", "Simple quasi-Newton ICA", "Picard"]
Ws  = [res_py[n]["Ws"] for n in names]
Ys  = [res_py[n]["Ys"] for n in names]
grads  = [res_py[n]["grads"][end] for n in names]
times  = [res_py[n]["times"][end] for n in names]
iters  = [res_py[n]["iters"][end] for n in names]

function loss_python(Y, W)
    n = size(Y, 2)
    M_storage4 = similar(Y)
    log_det = _logabsdet_any(W)   # your existing routine
    s_abs, s_tail = _log2cosh_sums_threaded!(M_storage4, Y)
    logcosh_sum = s_abs + 2*s_tail
    return -log_det + logcosh_sum / n
end

losses = [loss_python(Ys[i], Ws[i]) for i in eachindex(names)]

df_show = DataFrame(
    Method    = names,
    FinalGrad = [@sprintf("%.4e", g) for g in grads],
    FinalTime = [@sprintf("%.4f", t) for t in times],
    Finaliter = [@sprintf("%d", it) for it in iters],
    FinalLoss = [@sprintf("%.6f", ℓ) for ℓ in losses],
)

pretty_table(df_show)


┌─────────────────────────┬────────────┬───────────┬───────────┬───────────┐
│                  Method │  FinalGrad │ FinalTime │ Finaliter │ FinalLoss │
│                  String │     String │    String │    String │    String │
├─────────────────────────┼────────────┼───────────┼───────────┼───────────┤
│    Truncated Newton ICA │ 4.4537e-05 │    1.1783 │        22 │ 26.538976 │
│ Simple quasi-Newton ICA │ 1.3573e-04 │    0.5973 │        51 │ 26.538977 │
│                  Picard │ 2.0932e-04 │    0.5253 │        35 │ 26.538976 │
└─────────────────────────┴────────────┴───────────┴───────────┴───────────┘


## Singele precision

In [34]:
ml = pyimport("ml_ica")
root = "/home/xunjianli/Documents/Codes/Julia/ICAmm/faster-ica"
# matfile = joinpath(root, "examples", "eeg.mat")
matfile = joinpath(root, "examples", "fmri.mat")
res_py = py"run_ica_collect"(matfile, 1e-4, 250, false, "/tmp/ica_grad.png")  ## choose single precision

Dict{Any, Any} with 3 entries:
  "Simple quasi-Newton ICA" => Dict{Any, Any}("grads"=>[0.612867, 0.945988, 1.1…
  "Picard"                  => Dict{Any, Any}("grads"=>[0.612867, 0.945988, 0.7…
  "Truncated Newton ICA"    => Dict{Any, Any}("grads"=>[0.612867, 0.459176, 0.3…

In [35]:
names  = ["Truncated Newton ICA", "Simple quasi-Newton ICA", "Picard"]
Ws  = [res_py[n]["Ws"] for n in names]
Ys  = [res_py[n]["Ys"] for n in names]
grads  = [res_py[n]["grads"][end] for n in names]
times  = [res_py[n]["times"][end] for n in names]
iters  = [res_py[n]["iters"][end] for n in names]

function loss_python(Y, W)
    n = size(Y, 2)
    M_storage4 = similar(Y)
    log_det = _logabsdet_any(W)   # your existing routine
    s_abs, s_tail = _log2cosh_sums_threaded!(M_storage4, Y)
    logcosh_sum = s_abs + 2*s_tail
    return -log_det + logcosh_sum / n
end

losses = [loss_python(Ys[i], Ws[i]) for i in eachindex(names)]

df_show = DataFrame(
    Method    = names,
    FinalGrad = [@sprintf("%.4e", g) for g in grads],
    FinalTime = [@sprintf("%.4f", t) for t in times],
    Finaliter = [@sprintf("%d", it) for it in iters],
    FinalLoss = [@sprintf("%.6f", ℓ) for ℓ in losses],
)

pretty_table(df_show)


┌─────────────────────────┬────────────┬───────────┬───────────┬───────────┐
│                  Method │  FinalGrad │ FinalTime │ Finaliter │ FinalLoss │
│                  String │     String │    String │    String │    String │
├─────────────────────────┼────────────┼───────────┼───────────┼───────────┤
│    Truncated Newton ICA │ 4.4538e-05 │    1.1904 │        22 │ 26.538976 │
│ Simple quasi-Newton ICA │ 1.3500e-04 │    0.7477 │        51 │ 26.538977 │
│                  Picard │ 2.0763e-04 │    0.6108 │        35 │ 26.538976 │
└─────────────────────────┴────────────┴───────────┴───────────┴───────────┘
